# About
This notebook describes how centers were chosen for classifying activity in the BBP circuit. The github repo "Tribal Dynamics" (located at https://github.com/JasonPSmith/TriDy) is necessary to execute some of the cells in this notebook.

# Load packages

In [ ]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

## Step 1: Compute parameters
To choose the desired centers, parameter values must be computed at all the centers of the circuit. This is done in the external file `compute_parameters.py`, which is executed several times by the commands show below. These commands also appear in the file `compute_parameters.sh`. The resulting files are stored in the folder `parameters`.

In [ ]:
# python compute_parameters.py spectral
# python compute_parameters.py spectral_reverse
# python compute_parameters.py simplices
# python compute_parameters.py degrees
# python compute_parameters.py cc
# python compute_parameters.py dc
# python compute_parameters.py nbc
# python compute_parameters.py rc

Density paramaters are computed separately.

In [ ]:
rc = np.load('parameters/rc.npy')
simp0 = np.load('parameters/simp0.npy')
simp1 = np.load('parameters/simp1.npy')

# Reciprocal connections per nodes
rcpn = rc/simp0
np.save('parameters/rcpn.npy',rcpn)

# Reciprocal connections per edges
rcpe = rc/simp1
np.save('parameters/rcpe.npy',rcpe)

# Edges per nodes
epn = simp1/simp0
np.save('parameters/epn.npy',epn)

# The number of nodes in a neighbourhood is called `tribe_size` for legacy reasons
tribe_size = np.copy(simp0)
np.save('parameters/tribe_size.npy',tribe_size)

## Step 2: Set up containers for centers

In [ ]:
# Size of the excitatory-excitatory subcircuit (number of neurons)
nnum = 26567

# Number of neighborhoods to be used in classifying activity
number_nbhds = 50

# Percentage of centers to consider when sampling by density
density_cutoff = 0.01
cutoff_size = int(nnum*density_cutoff)

# Dictionary for holding center indices
data_dict = {'first_selection':[], 'second_selection':[], 'first_selection_order':[], 'second_selection_order':[]}
for i in range(number_nbhds):
        data_dict['chief'+str(i)] = []
        
# Computed parameters
filenames = [f for f in listdir('parameters/') if isfile(join('parameters/', f))]
paramnames = [f.split('.')[0] for f in filenames if f[-4:] == '.npy']
paramdata = {f:np.load('parameters/'+f+'.npy') for f in paramnames}

## Step 3.1: Random choices from the excitatory-excitatory subcircuit

In [ ]:
# Random selections    
choice_num = 4
print(f"Recording {choice_num} random groups of {number_nbhds}",flush=True)

for c in range(choice_num):
    data_dict['first_selection'].append(f"random{c}")
    data_dict['second_selection'].append(np.nan)
    data_dict['first_selection_order'].append(np.nan)
    data_dict['second_selection_order'].append(np.nan)
    current_indices = np.random.choice(range(nnum), size=number_nbhds, replace=False)
    for i in range(number_nbhds):
        data_dict['chief'+str(i)].append(current_indices[i])
        
print('All done', flush=True)

## Step 3.2: Random choices among the 1% sparsest and 1% densest

In [ ]:
# Random selections from 1% sparsest     
choice_num = 4
print(f"Recording {choice_num} random groups of {number_nbhds} from sparsest {density_cutoff}",flush=True)

values = np.argsort(rcpn)[:cutoff_size]
for c in range(choice_num):
    data_dict['first_selection'].append(f"sparse_random{c}")
    data_dict['second_selection'].append(np.nan)
    data_dict['first_selection_order'].append(np.nan)
    data_dict['second_selection_order'].append(np.nan)
    current_indices = np.random.choice(values, size=number_nbhds, replace=False)
    for i in range(number_nbhds):
        data_dict['chief'+str(i)].append(current_indices[i])

# Random selections from 1% densest     
choice_num = 4
print(f"Recording {choice_num} random groups of {number_nbhds} from densest {density_cutoff}",flush=True)

values = np.argsort(rcpn)[-cutoff_size:]
for c in range(choice_num):
    data_dict['first_selection'].append(f"dense_random{c}")
    data_dict['second_selection'].append(np.nan)
    data_dict['first_selection_order'].append(np.nan)
    data_dict['second_selection_order'].append(np.nan)
    current_indices = np.random.choice(values, size=number_nbhds, replace=False)
    for i in range(number_nbhds):
        data_dict['chief'+str(i)].append(current_indices[i])
        
print('All done', flush=True)

## Step 3.3: Single selection for all parameters

In [ ]:
# Every parameter, bottom value
print('Recording bottom values for all parameters', flush=True)
for p in paramnames:
    data_dict['first_selection'].append(p)
    data_dict['second_selection'].append(np.nan)
    data_dict['first_selection_order'].append('bottom')
    data_dict['second_selection_order'].append(np.nan)
    selection = np.argsort(paramdata[p])
    for i in range(number_nbhds):
        data_dict['chief'+str(i)].append(selection[i])
            
# Every parameter, top value
print('Recording top values for all parameters', flush=True)
for p in paramnames:
    data_dict['first_selection'].append(p)
    data_dict['second_selection'].append(np.nan)
    data_dict['first_selection_order'].append('top')
    data_dict['second_selection_order'].append(np.nan)
    selection = [x for _, x in sorted(zip(paramdata[p], range(nnum)))][::-1]
    for i in range(number_nbhds):
        data_dict['chief'+str(i)].append(selection[i])

print('All done', flush=True)

## Step 3.4: Double selection for all parameters
Restrict to the sparsest (then densest) 1% of neighbourhoods in the excitatory-excitatory subcircuit, then from that 1% select centers by their parameter value.

In [ ]:
# Double selection after restricting to sparsest neighbourhoods
print(f"Recording bottom and top values for all parameters, from sparsest {density_cutoff}", flush=True)
double_choices = np.argsort(paramdata['rc_per_nodes'])[:cutoff_size]

for p in paramnames:
    if p != 'rc_per_nodes':
        selection = [x for _, x in sorted(zip(paramdata[p][double_choices], double_choices))]
        
        # Bottom
        data_dict['first_selection'].append('rc_per_nodes')
        data_dict['second_selection'].append(p)
        data_dict['first_selection_order'].append('sparse')
        data_dict['second_selection_order'].append('bottom')
        for i in range(number_nbhds):
            data_dict['chief'+str(i)].append(selection[i])
        
        # Top
        data_dict['first_selection'].append(first_selection)
        data_dict['second_selection'].append(p)
        data_dict['first_selection_order'].append('sparse')
        data_dict['selection_order'].append('top')
        for i in range(number_nbhds):
            data_dict['chief'+str(i)].append(selection[::-1][i])
            
# Double selection after restricting to densest neighbourhoods
print(f"Recording bottom and top values for all parameters, from densest {density_cutoff}", flush=True)
double_choices = np.argsort(paramdata['rc_per_nodes'])[-cutoff_size:]

for p in paramnames:
    if p != 'rc_per_nodes':
        selection = [x for _, x in sorted(zip(paramdata[p][double_choices], double_choices))]
        
        # Bottom
        data_dict['first_selection'].append('rc_per_nodes')
        data_dict['second_selection'].append(p)
        data_dict['first_selection_order'].append('dense')
        data_dict['second_selection_order'].append('bottom')
        for i in range(number_nbhds):
            data_dict['chief'+str(i)].append(selection[i])
        
        # Top
        data_dict['first_selection'].append(first_selection)
        data_dict['second_selection'].append(p)
        data_dict['first_selection_order'].append('dense')
        data_dict['selection_order'].append('top')
        for i in range(number_nbhds):
            data_dict['chief'+str(i)].append(selection[::-1][i])
            
print('All done', flush=True)

## Step 4: Export final dataframe

In [ ]:
# Construct dataframe
df = pd.DataFrame.from_dict(data_dict)

# Export dataframe
df.to_pickle('selections.pkl')

# Export ragged array for TriDy-tools
partition = []
for i in range(df.shape[0]):
    current = []
    for j in range(number_nbhds):
        current.append(exc_loc[df.iloc[i]['chief'+str(j)]])
    partition.append(current)
np.save('TriDy-tools/bins/partition_reliability.npy', np.array(partition))

print('All done', flush=True)